這個 notebook 的資料集來自 Kaggle 上的 Speed Dating Experiment Dataset (快速配對實驗資料集)

我們可以訓練一個簡單的邏輯回歸模型（一種線性模型）

藉由觀察模型中的權重，去挖掘怎樣的「模式」

能夠更（容易/不容易）約到第二次約會

In [ ]:
%matplotlib inline
import random
random.seed(10)
import numpy as np 
np.random.seed(10)
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
import multiprocessing
cpn_cnt = multiprocessing.cpu_count()


第一步，先觀察一下資料：

In [ ]:
DATA = pd.read_csv('../input/Speed Dating Data.csv', encoding='ISO-8859-1')
DATA.head()

樣本數： 8378 個人

In [ ]:
# print('features: ', list(DATA))
print('samples: ', len(DATA.iloc[:,0]))

In [ ]:
# DATA.isnull().sum()

ID 和 IID 只是資料裡人們的編號，不是很重要

In [ ]:
DATA.drop(['id', 'iid'], axis=1, inplace=True)

來統計一下成功/沒有成功配對的數量

In [ ]:
pd.crosstab(DATA['match'], 'count')

觀察資料中的 feature 名稱，對照敘述檔 (Speed Dating Data Key.doc) 找到我們有興趣的 feature

In [ ]:
filter_1 = ['gender', 'match', 'int_corr', 'age_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age', 'imprace', 'imprelig', 'income', 'goal', 'date', 'go_out', 'career', 'career_c', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', 'expnum', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1', 'intel3_1', 'amb3_1', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1', 'dec', 'attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'prob', 'met', 'match_es', 'attr1_s', 'sinc1_s', 'intel1_s', 'fun1_s', 'amb1_s', 'shar1_s', 'attr3_s', 'sinc3_s', 'intel3_s', 'fun3_s', 'amb3_s', 'satis_2', 'length', 'numdat_2', 'attr7_2', 'sinc7_2', 'intel7_2', 'fun7_2', 'amb7_2', 'shar7_2', 'attr1_2', 'sinc1_2', 'intel1_2', 'fun1_2', 'amb1_2', 'shar1_2', 'attr4_2', 'sinc4_2', 'intel4_2', 'fun4_2', 'amb4_2', 'shar4_2', 'attr2_2', 'sinc2_2', 'intel2_2', 'fun2_2', 'amb2_2', 'shar2_2', 'attr3_2', 'sinc3_2', 'intel3_2', 'fun3_2', 'amb3_2', 'attr5_2', 'sinc5_2', 'intel5_2', 'fun5_2', 'amb5_2', 'you_call', 'them_cal', 'date_3', 'numdat_3', 'num_in_3', 'attr1_3', 'sinc1_3', 'intel1_3', 'fun1_3', 'amb1_3', 'shar1_3', 'attr7_3', 'sinc7_3', 'intel7_3', 'fun7_3', 'amb7_3', 'shar7_3', 'attr4_3', 'sinc4_3', 'intel4_3', 'fun4_3', 'amb4_3', 'shar4_3', 'attr2_3', 'sinc2_3', 'intel2_3', 'fun2_3', 'amb2_3', 'shar2_3', 'attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3', 'attr5_3', 'sinc5_3', 'intel5_3', 'fun5_3', 'amb5_3']
DATA = DATA[filter_1]
for f in sorted(list(DATA)):
    print(f)

觀察 prefix 重複的 feature

下面使用 corrlation matrix 來觀察 feature 間是否高度相關

若有幾種 feature 高度相關，代表 feature 間很類似

我們只留下其中一種當 feature

In [ ]:
ambs = '''amb
amb1_1
amb1_2
amb1_3
amb1_s
amb2_1
amb2_2
amb2_3
amb3_1
amb3_2
amb3_3
amb3_s
amb4_1
amb4_2
amb4_3
amb5_1
amb5_2
amb5_3
amb7_2
amb7_3
amb_o'''.split()

attrs = '''attr
attr1_1
attr1_2
attr1_3
attr1_s
attr2_1
attr2_2
attr2_3
attr3_1
attr3_2
attr3_3
attr3_s
attr4_1
attr4_2
attr4_3
attr5_1
attr5_2
attr5_3
attr7_2
attr7_3
attr_o'''.split()

funs = '''fun
fun1_1
fun1_2
fun1_3
fun1_s
fun2_1
fun2_2
fun2_3
fun3_1
fun3_2
fun3_3
fun3_s
fun4_1
fun4_2
fun4_3
fun5_1
fun5_2
fun5_3
fun7_2
fun7_3
fun_o'''.split()

intels = '''intel
intel1_1
intel1_2
intel1_3
intel1_s
intel2_1
intel2_2
intel2_3
intel3_1
intel3_2
intel3_3
intel3_s
intel4_1
intel4_2
intel4_3
intel5_1
intel5_2
intel5_3
intel7_2
intel7_3
intel_o'''.split()

shars = '''shar
shar1_1
shar1_2
shar1_3
shar1_s
shar2_1
shar2_2
shar2_3
shar4_1
shar4_2
shar4_3
shar7_2
shar7_3
shar_o'''.split()

sincs = '''sinc
sinc1_1
sinc1_2
sinc1_3
sinc1_s
sinc2_1
sinc2_2
sinc2_3
sinc3_1
sinc3_2
sinc3_3
sinc3_s
sinc4_1
sinc4_2
sinc4_3
sinc5_1
sinc5_2
sinc5_3
sinc7_2
sinc7_3
sinc_o'''.split()

In [ ]:
intersted_field = [ambs, attrs, funs, intels, shars, sincs]

In [ ]:
for f in intersted_field:
    fig, ax = plt.subplots(dpi=90)
    corr = DATA[f].corr()
    sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values, ax=ax)
    plt.show()

選擇 feature

In [ ]:
others = '''age
age_o
art
career_c
clubbing
concerts
date
date_3
dec
dec_o
dining
exercise
exphappy
expnum
gaming
gender
go_out
goal
hiking
imprace
imprelig
income
int_corr
length
like
like_o
match_es
met
met_o
movies
museums
music
num_in_3
numdat_2
numdat_3
pf_o_amb
pf_o_att
pf_o_fun
pf_o_int
pf_o_sha
pf_o_sin
prob
prob_o
reading
satis_2
shopping
sports
theater
them_cal
tv
tvsports
yoga
you_call'''.split()

filter_2 = ['match', 'amb', 'amb1_1', 'amb2_1', 'amb3_1', 'amb4_1', 'amb5_1', 'amb7_2', 'amb_o', 
'attr', 'attr1_1', 'attr2_1', 'attr3_1', 'attr4_1', 'attr5_1', 'attr7_2', 'attr_o',
'fun', 'fun1_1', 'fun2_1', 'fun3_1', 'fun4_1', 'fun5_1', 'fun7_2', 'fun_o',
'intel', 'intel1_1', 'intel2_1', 'intel2_3', 'intel3_1', 'intel4_1', 'intel5_1', 'intel7_2', 'intel_o',
'sinc', 'sinc1_1', 'sinc2_1', 'sinc2_3', 'sinc3_1', 'sinc4_1', 'sinc5_1', 'sinc7_2', 'sinc_o'] + shars + others

檢查、刪除缺失值

In [ ]:
DATA = DATA[filter_2]
DATA.isnull().sum()

In [ ]:
DATA = DATA.iloc[:, np.asarray(DATA.isnull().sum()<1000, dtype=np.bool)]
DATA.isnull().sum()

feature 與「成功配對」的相關性 (correlation)

In [ ]:
# corrlations with match
corr = DATA.corrwith(DATA['match'])
corr.sort_values(ascending=False)

In [ ]:
neg = np.abs(corr)<0.01
black_list = list(corr[neg].keys())
black_list

feature 名稱中，如果帶有 

*_o

的後綴，代表這是對方對自己的評價

而 dec_o 這一項，代表對方口頭答應下一次約會

但實際情況不太可能知道 dec_o

所以把 dec_o 刪除

另外，我們對對方的職業種類沒興趣，因為比較難量化

如果要用職業種類作為 feature 、訓練模型，

較推薦使用決策樹

In [ ]:
### We don't know potential parter's decision in real world. 

black_list += ['dec_o'] 
black_list += ['career_c', 'length'] # not interested in career and length of night event

In [ ]:
DATA.drop(black_list, axis=1, inplace=True)
for key, val in DATA.dtypes.items():
    print('{:>10}: {:s}'.format(str(key), str(val)))

刪除缺失值

In [ ]:
DATA.dropna(inplace=True)
DATA.isnull().sum()

最後剩餘樣本數： 5567 人

In [ ]:
print('samples: ', len(DATA.iloc[:,0]))

做一下 ANOVA 觀察 "match" 可能受哪些因素影響

如果某變數顯著 (p<0.05)，我們在表格最右側標上 '\*' 號

In [ ]:
formula = 'match ~ amb*amb_o + attr*attr_o + fun*fun_o + intel*intel_o + sinc*sinc_o + C(shar1_1) + age*age_o + clubbing + date + dining + go_out + sports + int_corr + like*like_o + met*met_o + movies + museums + music + numdat_2 + prob*prob_o + reading + satis_2 + tv + yoga + gaming + goal + C(met)*C(met_o)' # 觀察變數
lm_model = ols(formula, DATA).fit()
aov_table = sm.stats.anova_lm(lm_model, typ=2)
significat_fators = aov_table['PR(>F)']<0.05
aov_table['significant'] = np.where(significat_fators, '*', ' ')
display(aov_table)

下面的表格只列出顯著的變數

In [ ]:
display(aov_table[significat_fators]) # 

從上述表格可以看出：

1. 有共同愛好
2. 自己有企圖心 / 對方認為你有企圖心
3. 自己覺得自己有吸引力 / 對方覺得你對他來說有吸引力
4. 自己覺得自己是有趣的人 / 對方覺得你是有趣的人
5. 自己覺得自己很真誠 / 對方覺得你很真誠
6. 自己有參與社團的興趣
7. 自己有運動的興趣
8. 你喜歡他 / 他喜歡你
9. 你覺得"有機會" / 他覺得"有機會"
10. 你對這次約會的在意程度

這幾個因素有可能跟最後配對(match)成功/失敗有關係

但是其中有幾項由於變數間交互作用對影響 match 是顯著的，例如：

1. 自己覺得自己有吸引力 * 對方覺得你對他來說有吸引力
2. 你喜歡他 * 他喜歡你
3. 你覺得"有戲" * 他覺得"有戲"

這些變數的交互作用可能影響 match 結果顯著，所以這些變數不能單獨拆開來看，要考慮到變數間的交互作用對最後 match 的影響。

最後要強調，"顯著" 並不代表一定對 "match" 有影響，
就算有影響，也看不出是好是壞。接下來我們訓練一個簡單的邏輯回歸模型，看看有哪些因素 "可能" 對 "match" 有正向/負面影響。


In [ ]:
X, Y = np.array(DATA.iloc[:, 1:], dtype=np.float32), np.array(DATA.iloc[:, 0], dtype=np.int16)
print(X.shape)
print(Y.shape)

模型訓練

In [ ]:
random.seed(10) # 固定變數，讓結果可以重現
np.random.seed(10)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from IPython.display import display

In [ ]:
parameters = {'C': [0.1, 1, 10], 'max_iter': [500, 1000], 'solver': ['lbfgs', 'liblinear']}

In [ ]:
X, X_test, Y, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=True, stratify=Y)
print('{:d} samples for train/val, {:d} samples for testing.'.format(len(X), len(X_test)))

lr = GridSearchCV(LogisticRegression(), param_grid=parameters, cv=10, scoring='accuracy', n_jobs=max(1, cpn_cnt-1))
lr.fit(X, Y)
display(lr.cv_results_)

In [ ]:
display(lr.best_params_)

In [ ]:
print('Testing set performance: ')
preds = lr.predict(X_test) # prediction
acc = accuracy_score(Y_test, preds) # evaluations
precision = precision_score(Y_test, preds)
recall = recall_score(Y_test, preds)
f1 = f1_score(Y_test, preds)
print('acc: {:.2f}, precision: {:.2f}, recall: {:.2f}, f1: {:.2f}'.format(acc, precision, recall, f1))

底下印出模型權重

模型權重預高，代表對應的 feature 愈能句提高成功配對的機率

反之權重愈低的 feature 愈降低成功配對的機率

In [ ]:
best_lr = lr.best_estimator_
W_inspect = np.append(best_lr.coef_.flatten(), best_lr.intercept_.flatten(), axis=-1) # Check weights of perceptron to acquire knowledge of dating? ;)
features_key = np.array(list(DATA.iloc[:, 1:]) + ['w0 (+1)'])
order = np.argsort(-W_inspect)
weights, keys = W_inspect[order], features_key[order]
for w, k in zip(weights, keys):
    print('{:>10}: {:.4f}'.format(k, w))

觀察資料，最高權重的幾個 feature 為：

1. dec: 自己想約下一次
2. like_o: 對方喜歡自己
3. attr_o: 對方覺得自己很有魅力
4. prob_o: 對方覺得 "有機會"
5. int_corr: 雙方興趣相近
6. prob: 自己覺得 "有機會"
7. fun_o: 對方覺得自己有趣
8. art: 自己喜歡藝術類的東西
9. intel: 自己覺得自己聰明
10. hiking: 喜歡爬山
11. tv: 喜歡看電視
12. sinc: 自己態度很真誠
13. intel_o: 對方覺得自己聰明

...

權重最低、而且是負的幾個 feature:

1. met_o: 對方在約會前認識你（驚）
2. amb_o: 對方覺得你很有野心
3. sinc_o: 對方覺得你很真誠（驚）
4. attr: 自己覺得自己很有魅力（驚）
5. numdat_2: 參加 "快速配對" 類活動的次數
6. date: 愈高代表平常約會頻率預低（換句話說：平常沒有約會的人，比較難成功約到下一次約會）
7. satis_2: 你對於這次約會對象多滿意
8. shar1_1: 有共同愛好
9. gaming: 喜歡打遊戲
10. movies: 喜歡看電影
11. goal: 愈高代表對這次約會的結果愈認真、愈在意
12. age: 年齡
13. age_o: 對方年齡

...

跑出來的結果非常有趣，之後也許可以嘗試決策樹

就不會受限於線性模型這樣類似加減分的性質

也可以更好地觀察潛在的模式